# New facility location selection

#### Jeffrey G Dupree

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results](#results)
* [Discussion](#discussion)
* [Conclusion](#conclusion)

## I. Introduction: Business Problem <a name="introduction"></a>

The owner of several successful gyms wants to open a new gym in Tampa, FL. They want to ensure that the gym’s location is in an area not already saturated with gyms and is close to other businesses. This will reduce competition and increase use by the employees of the nearby businesses. This initial analysis will be to determine to a neighborhood level, where to place the new gym facility. Later analysis and research of available real estate will be required to select the final location. That is beyond the scope of this analysis.

## II. Data <a name="data"></a>

The primary data for this analysis will be location and business data from Foursquare.

ZIP Codes for Tampa, FL were sourced from the United States Postal Service (USPS) website “www.usps.com”. ZIP Codes associated with specific Post Office (PO) Boxes, companies, or organization and not areas of Tampa, FL were omitted.

Using the Foursquare application programming interface (API) business type and frequency of occurrence will be extracted for each ZIP code area. The focus will be on classifying ZIP code areas based on the lack of gyms and concentration of businesses that commonly have a large workforce.

## III. Methodology <a name="methodology"></a>
The main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

## IV. Results <a name="results"></a>

## V. Discussion <a name="discussion"></a>
Discuss any observations you noted and any recommendations you can make based on the results.

## VI. Conclusion <a name="conclusion"></a>